# **1-1. 통합 테스트**

## **1.환경설정**

### (1) 경로 설정

로컬 경로 설정

In [1]:
path = './'

### (2) 라이브러리

In [2]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import torch
import sys
sys.path.append(path)

from transformers import AutoTokenizer, AutoModelForSequenceClassification


# 더 필요한 라이브러리 추가 -------------
import emergencymy as em

## **2. 단위 테스트**

* 세부사항 : 아래 단계별로 데이터가 순차적으로 처리되도록 단위 테스트를 진행합니다.

### (1) open ai key 등록

In [3]:
openai.api_key = em.load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key
print(os.environ['OPENAI_API_KEY'][:30])

sk-proj-yefXrQbXMhH2ZW3i47_B4e


### (2) audio to text

In [4]:
audio_path = path + 'audio/'
filename = 'audio1.mp3'
result = em.audio2text(audio_path, filename)
result

'지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어요. 근데 조금 어지럽다고 하네요. 네네 계단에서 굴렀어요. 지금은 물 마시고 있는데 이거 응급실로 가봐야 할까요? 피도 지금 머졌어요. 네네 나이는 마흔아홉 살 이세요. 어떻게 해야 할지 모르겠어요.\n'

### (3) text summary

In [5]:
result = em.text2summary(result)
result

'남자 어르신이 계단에서 굴러 넘어져 머리에서 피가 나고 어지러워하는 상황. 응급실 방문이 필요하며 신속한 처치가 필요함., 넘어짐, 피, 어지러움'

### (4) 응급실 등급분류

In [6]:
# 모델, 토크나이저 로드
save_directory = path + "fine_tuned_bert_my"
model = AutoModelForSequenceClassification.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)

In [7]:
predicted_class, probabilities = em.predict(result, model, tokenizer)

print(f"예측된 클래스: {predicted_class}")
print(f"예측된 클래스 이름: {predicted_class+1}등급")
print(f"클래스별 확률: {probabilities}")

예측된 클래스: 1
예측된 클래스 이름: 2등급
클래스별 확률: tensor([[0.1206, 0.7857, 0.0339, 0.0050, 0.0548]])


### (5) 응급실추천

In [10]:
emergency = pd.read_csv(path + '응급실 정보.csv')

map_key = em.load_file(path + 'map_key.txt')
map_key = json.loads(map_key)
# c_id, c_key = 'hqev5yhpp0', '0hKV6LdcBYambAIlHkVtAMuxCuRD6ypCVQrSGjne'
c_id, c_key = map_key['c_id'], map_key['c_key']

In [11]:
predicted_class = 2
start_lat, start_lng = 37.358877, 127.114863

emergency_list = emergency.to_dict('records')

if predicted_class <= 2 :
    result = em.recommend_hospital3(emergency_list, start_lat, start_lng, c_id, c_key)

result

검색 범위 확장 중: 5.50km * 5.50km


,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도,거리(km)
0,분당서울대학교병원,"경기도 성남시 분당구 구미로173번길 82 (구미동, 분당서울대학교병원)",권역응급의료센터,031-787-2114,031-787-3119,37.352026,127.124484,2.778
1,대진의료재단분당제생병원,경기도 성남시 분당구 서현로180번길 20 (서현동),지역응급의료센터,031-779-0114,031-779-0119,37.387871,127.121328,5.018
2,국군수도병원,경기도 성남시 분당구 새마을로177번길 81 (율동),지역응급의료기관,031-725-6075,031-725-6119,37.391867,127.148586,6.125


## **3. 통합 테스트**

* 세부사항
    * [2. 단위 테스트] 의 내용을 순차적으로 실행하며 결과를 확인합니다.

In [14]:
# 1. 준비 작업 ---------------------------------------------
openai.api_key = em.load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

map_key = em.load_file(path + 'map_key.txt')
map_key = json.loads(map_key)
# c_id, c_key = 'hqev5yhpp0', '0hKV6LdcBYambAIlHkVtAMuxCuRD6ypCVQrSGjne'
c_id, c_key = map_key['c_id'], map_key['c_key']


emergency = pd.read_csv(path + '응급실 정보.csv')

# 모델, 토크나이저 로드
save_directory = path + "fine_tuned_bert_my"
model = AutoModelForSequenceClassification.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)


# 2. 데이터 처리(pipeline) ---------------------------------
# input
audio_path = path + 'audio/'
filename = 'audio1.mp3'

# 처리
result = em.audio2text(audio_path, filename)
result = em.text2summary(result)
predicted_class, _ = em.predict(result, model, tokenizer)

# 테스트 input --------------------
predicted_class = 2  # 테스트용
start_lat, start_lng = 37.358877, 127.114863
# ---------------------------------
emergency_list = emergency.to_dict('records')
if predicted_class <= 2 :
    result = em.recommend_hospital3(emergency_list, start_lat, start_lng, c_id, c_key)
    display(result)
else :
    print('개인 건강관리')

검색 범위 확장 중: 5.50km * 5.50km


,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도,거리(km)
0,분당서울대학교병원,"경기도 성남시 분당구 구미로173번길 82 (구미동, 분당서울대학교병원)",권역응급의료센터,031-787-2114,031-787-3119,37.352026,127.124484,2.778
1,대진의료재단분당제생병원,경기도 성남시 분당구 서현로180번길 20 (서현동),지역응급의료센터,031-779-0114,031-779-0119,37.387871,127.121328,4.941
2,국군수도병원,경기도 성남시 분당구 새마을로177번길 81 (율동),지역응급의료기관,031-725-6075,031-725-6119,37.391867,127.148586,6.125
